In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -ls /tmp
!hdfs dfs -mkdir /tmp/hivep12
!hdfs dfs -ls /tmp/
!hdfs dfs -copyFromLocal data.tsv /tmp/hivep12/

Found 6 items
drwxrwx---   - root supergroup          0 2020-02-11 03:08 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-11 03:10 /tmp/hive
drwxr-xr-x   - root supergroup          0 2020-02-11 04:12 /tmp/hivep10
drwxr-xr-x   - root supergroup          0 2020-02-11 04:24 /tmp/hivep11
drwxr-xr-x   - root supergroup          0 2020-02-11 03:16 /tmp/hivep8
drwxr-xr-x   - root supergroup          0 2020-02-11 03:54 /tmp/hivep9
Found 7 items
drwxrwx---   - root supergroup          0 2020-02-11 03:08 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-11 03:10 /tmp/hive
drwxr-xr-x   - root supergroup          0 2020-02-11 04:12 /tmp/hivep10
drwxr-xr-x   - root supergroup          0 2020-02-11 04:24 /tmp/hivep11
drwxr-xr-x   - root supergroup          0 2020-02-11 04:28 /tmp/hivep12
drwxr-xr-x   - root supergroup          0 2020-02-11 03:16 /tmp/hivep8
drwxr-xr-x   - root supergroup          0 2020-02-11 03:54 /tmp/hivep9


In [3]:
%%hive
DROP TABLE IF EXISTS t0;
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;

DROP TABLE IF EXISTS t0;
OK
Time taken: 19.281 seconds
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
OK
Time taken: 4.135 seconds
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;
Loading data to table default.t0
OK
Time taken: 3.433 seconds


In [7]:
%%hive
DROP TABLE IF EXISTS pt12;
CREATE TABLE pt12
AS
SELECT 
letra1 AS c1,
letra  AS c2,
COUNT(*) AS c3
FROM t0 
LATERAL VIEW explode(c2) adTable1 AS letra1
LATERAL VIEW explode(c3) adTable AS letra,numero
GROUP BY letra1,letra;

INSERT OVERWRITE DIRECTORY '/tmp/hivep12'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
SELECT * FROM pt12;

SELECT * FROM pt12;
Query ID = root_20200211043408_b076353f-338c-4d34-8d2f-e3da95630bd1
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1581390426457_0014, Tracking URL = http://714a45eef6db:8088/proxy/application_1581390426457_0014/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1581390426457_0014
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-02-11 04:34:24,176 Stage-1 map = 0%,  reduce = 0%
2020-02-11 04:34:35,065 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 3.07 sec
MapReduce Total cumulative CPU time: 3 seconds 70 msec
Ended Job = job_1581390426457_0014
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/tmp/hivep12/.hive-staging_hive_2020-02-11_04-34-08_438_5708915029910573769-1/-ext-10000
Moving data to directory /tmp/hi

In [8]:
!hdfs dfs -cat /tmp/hivep12/000000_0 

a,aaa,5
a,bbb,7
a,ccc,13
a,ddd,13
a,eee,7
a,fff,10
a,ggg,8
a,hhh,8
a,iii,7
a,jjj,10
b,aaa,4
b,bbb,7
b,ccc,5
b,ddd,7
b,eee,5
b,fff,8
b,ggg,4
b,hhh,7
b,iii,7
b,jjj,5
c,aaa,5
c,bbb,4
c,ccc,12
c,ddd,9
c,eee,6
c,fff,8
c,ggg,7
c,hhh,7
c,iii,8
c,jjj,8
d,aaa,4
d,bbb,6
d,ccc,7
d,ddd,5
d,eee,6
d,fff,8
d,ggg,6
d,hhh,4
d,iii,9
d,jjj,8
e,aaa,3
e,bbb,8
e,ccc,9
e,ddd,7
e,eee,7
e,fff,9
e,ggg,4
e,hhh,4
e,iii,8
e,jjj,3
f,aaa,8
f,bbb,10
f,ccc,13
f,ddd,17
f,eee,11
f,fff,11
f,ggg,9
f,hhh,10
f,iii,10
f,jjj,12
g,aaa,3
g,bbb,3
g,ccc,6
g,ddd,5
g,eee,4
g,fff,5
g,ggg,4
g,hhh,3
g,iii,4
g,jjj,6


In [9]:
!hdfs dfs -copyToLocal /tmp/hivep12/000000_0 output;

In [22]:
%%hive
DROP TABLE pto12;
DROP TABLE t0;

DROP TABLE pto11;
OK
Time taken: 0.467 seconds
DROP TABLE t0;
OK
Time taken: 0.846 seconds


In [10]:
!rm *.tsv *.log

rm: cannot remove 'derby.log': Text file busy


In [11]:
%hive_quit